In [ ]:
#Creating a MODULE for later use of the logistic model - getting the ML learning ready for deployment

import pandas as pd
import numpy as np
import pickle

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator , TransformerMixin

from sklearn.linear_model import LogisticRegression
from sklearn import metrics


In [ ]:
#CUSTOM SCALER CLASS

class CustomScaler(BaseEstimator,TransformerMixin):
    
    def __init__(self,columns,copy=True,with_mean=True,with_std=True):
        self.scaler = StandardScaler(copy,with_mean,with_std)
        self.columns = columns
        self.mean_ = None
        self.var_ = None
        
    def fit(self,X, y = None):
        self.scaler.fit(X[self.columns], y)
        self.mean_ = np.mean(X[self.columns])
        self.var_ =np.var(X[self.columns])
        
    def transform(self,X , y=None, copy=None):
        init_col_order = X.columns
        X_scaled = pd.DataFrame(self.scaler.transform(X[self.columns]), columns =self.columns)
        X_not_scaled = X.loc[:,~X.columns.isin(self.columns)]
        return pd.concat([X_not_scaled, X_scaled], axis=1)[init_col_order]

In [3]:

#storing code in a module will allow us to resue it without trouble


#create the special class that we are going to use from here on to predict data

class absenteeism_model():
    def __init__(self,model_file,scaler_file):
    #read the 'model' and ' scaler' files which were saved
        with open('model','rb') as model_file, open('scaler','rb') as scaler_file:
            self.reg = pickle.load(model_file)
            self.scaler = pickle.load(scaler_file)
            self.data = None
            
    
    #take a data file (*.csv) and preprocessed it in the same 
    def load_and_clean_data(self,data_file):
        
        #import the data
        df = pd.read_csv(data_file,delimiter-',')
       
        # store the data in a new variable for later use
        self.df_with_predictions = df.copy()
        
        # drop the "ID" column
        df = df.drop(['ID'], axis=1)
        
        #to preserve the code we have created in the previous section, we will add a column with 'Nan' strings
        df['Absenteeism Time in Hours'] = 'Nan'
        
        #create a seperate dataframe, containing dummy values for ALL available reasons
        reason_columns = pd.get_dummies(df['Reason for Absence'], drop_first = True)
        
        
        #split reason_columns into 4 types
        reason_type_1 = reason_colunms.loc[:,1:14].max(axis=1)
        reason_type_1 = reason_colunms.loc[:,15:17].max(axis=1)
        reason_type_1 = reason_colunms.loc[:,18:21].max(axis=1)
        reason_type_1 = reason_colunms.loc[:,22:].max(axis=1)